In [1]:
from IPython.display import Image

In [2]:
# 크롤링 관련 모듈
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time

In [3]:
driver=None

In [4]:
# 드라이버 세팅 함수
def set_chrome_driver():
    global driver
    if driver is None:
        chrome_options = webdriver.ChromeOptions()
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    else:
        return driver

In [5]:
# 드라이버 리셋 함수
def reset_driver():
    global driver
    driver.quit()
    if driver is None:
        pass
    else:
        driver = None

In [6]:
# 드라이버 세팅
driver=set_chrome_driver()

In [7]:
# db 접근
import pymysql

In [8]:
 conn = pymysql.connect(
     host='18.117.197.254', 
     user='ssossotable', 
     password='Mysql7968!', 
     db='ssossotable_food', 
     charset='utf8')

In [9]:
foodList = []
imageSrcList = []

In [10]:
with conn.cursor() as cursor1:
    sql = "SELECT name FROM food ORDER BY id"
    cursor1.execute(sql)
    result = cursor1.fetchall()
    for food in result:
        foodList.append(food[0])

In [66]:
for i in range(len(imageSrcList), len(foodList)):
    URL="https://www.google.com/search?q="+foodList[i]+"&hl=ko&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjuu9Lk1sP7AhU4h1YBHWpxA9IQ_AUoAXoECAIQAw&biw=1109&bih=854&dpr=1.5"
    # 타겟 페이지 접속
    driver.get(URL)
    time.sleep(2)
    driver.find_element(By.XPATH, 
        "//*[@id='islrg']/div[1]/div[1]/a[1]/div[1]/img").click()
    time.sleep(1)
    e = driver.find_elements(By.CLASS_NAME, 'KAlRDb')
    time.sleep(2)
    image = e[0].get_dom_attribute('src')
    imageSrcList.append(image)

오류 날 경우

In [65]:
URL="https://www.google.com/search?q="+foodList[len(imageSrcList)]+"&hl=ko&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjuu9Lk1sP7AhU4h1YBHWpxA9IQ_AUoAXoECAIQAw&biw=1109&bih=854&dpr=1.5"
# 타겟 페이지 접속
driver.get(URL)
time.sleep(2)
driver.find_element(By.XPATH, 
    "//*[@id='islrg']/div[1]/div[2]/a[1]/div[1]/img").click()
time.sleep(1)
e = driver.find_elements(By.CLASS_NAME, 'KAlRDb')
time.sleep(2)
image = e[0].get_dom_attribute('src')
imageSrcList.append(image)

In [58]:
imageSrcList.append("http://ssossotable.com/src/food_placeholder.png")

update 문

In [67]:
with conn.cursor() as cursor2:
    i = 0
    for src in imageSrcList:
        sql = "UPDATE food SET image_yhj = %s WHERE name = %s"
        cursor2.execute(sql, (src, foodList[i]))
        conn.commit()
        i+=1